Aim of this project is to conduct an initial data exploration and data preparation, perform at least 1 Association Rule analysis and provide detailed interpretation of the findings

In [1]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=e1f2c47cc8d517d6551b5d4d11aa3486728a30c37d111ca1c7a25425a7ca6a06
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from apyori import apriori

In [3]:
data = pd.read_excel("/content/Online Retail.xlsx")

In [4]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,551429,10002,NaN,-3,2011-04-28 15:05:00,0.00,NaN,United Kingdom
1,550452,10002,INFLATABLE POLITICAL GLOBE,1,2011-04-18 12:56:00,0.85,14525.0,United Kingdom
2,550272,10002,INFLATABLE POLITICAL GLOBE,62,2011-04-15 12:14:00,0.85,18079.0,United Kingdom
3,548714,10002,INFLATABLE POLITICAL GLOBE,2,2011-04-03 15:07:00,0.85,17337.0,United Kingdom
4,548702,10002,INFLATABLE POLITICAL GLOBE,4,2011-04-03 11:36:00,0.85,17085.0,United Kingdom


In [5]:
data.shape

(541909, 8)

In [6]:
#Getting info from column types and entries
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
#Count null values
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [9]:
#dropping customer id due to unique id and missing values
data.drop(["CustomerID"],axis=1, inplace=True)

In [10]:
#Looking at cancelled invoice
data[data["InvoiceNo"].str.startswith('C', na=False)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
221,C558716,10133,COLOURING PENCILS BROWN TUBE,-10,2011-07-01 13:22:00,0.42,United Kingdom
422,C571707,10135,COLOURING PENCILS BROWN TUBE,-1,2011-10-18 15:33:00,1.25,United Kingdom
602,C575257,11001,ASSTD DESIGN RACING CAR PEN,-1,2011-11-09 11:59:00,1.69,United Kingdom
603,C569682,11001,ASSTD DESIGN RACING CAR PEN,-2,2011-10-05 14:33:00,1.69,United Kingdom
604,C568412,11001,ASSTD DESIGN RACING CAR PEN,-1,2011-09-27 10:54:00,1.69,United Kingdom
...,...,...,...,...,...,...,...
541902,C544580,S,SAMPLES,-1,2011-02-21 14:25:00,20.55,United Kingdom
541903,C544580,S,SAMPLES,-1,2011-02-21 14:25:00,29.99,United Kingdom
541904,C544580,S,SAMPLES,-1,2011-02-21 14:25:00,9.74,United Kingdom
541905,C537581,S,SAMPLES,-1,2010-12-07 12:03:00,12.95,United Kingdom


In [11]:
n_data = data[~data['InvoiceNo'].str.startswith('C', na=False)]
n_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country
0,551429,10002,NaN,-3,2011-04-28 15:05:00,0.00,United Kingdom
1,550452,10002,INFLATABLE POLITICAL GLOBE,1,2011-04-18 12:56:00,0.85,United Kingdom
2,550272,10002,INFLATABLE POLITICAL GLOBE,62,2011-04-15 12:14:00,0.85,United Kingdom
3,548714,10002,INFLATABLE POLITICAL GLOBE,2,2011-04-03 15:07:00,0.85,United Kingdom
4,548702,10002,INFLATABLE POLITICAL GLOBE,4,2011-04-03 11:36:00,0.85,United Kingdom


In [12]:
n_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532621 entries, 0 to 541908
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    532621 non-null  object        
 1   StockCode    532621 non-null  object        
 2   Description  531167 non-null  object        
 3   Quantity     532621 non-null  int64         
 4   InvoiceDate  532621 non-null  datetime64[ns]
 5   UnitPrice    532621 non-null  float64       
 6   Country      532621 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 32.5+ MB


In [13]:
n_data.shape

(532621, 7)

In [16]:
#group by country
country_data = n_data.groupby(['Country']).agg(
                     InvoiceCount = ('InvoiceNo', 'count'),
                     QuantityCount = ('Quantity', 'count'),
                     TotalPrice = ('UnitPrice', 'sum')).reset_index()

In [17]:
#Sorting the values
country_data.sort_values('TotalPrice', ascending=False, inplace=True, ignore_index=True)
country_data

,Country,InvoiceCount,QuantityCount,TotalPrice
0,United Kingdom,487622,487622,1845443.914
1,EIRE,7894,7894,38489.950
2,France,8408,8408,36992.790
3,Germany,9042,9042,33532.140
4,Singapore,222,222,12949.990
5,Spain,2485,2485,9504.340
6,Portugal,1501,1501,8770.720
7,Belgium,2031,2031,7372.850
8,Hong Kong,284,284,6666.740
9,Switzerland,1967,1967,6634.360


After sorting the values, I can infer that the top purchaser is from United Kingdom, with a total invoice count of 487,622 and is the largest source of revenue. 

In [18]:
#Taking a look at the products bought
n_data['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER     2327
JUMBO BAG RED RETROSPOT                2115
REGENCY CAKESTAND 3 TIER               2019
PARTY BUNTING                          1707
LUNCH BAG RED RETROSPOT                1594
                                       ... 
LASER CUT MULTI STRAND NECKLACE           1
FILIGREE DIAMANTE CHAIN                   1
samples                                   1
ASSORTED COLOUR SILK GLASSES CASE         1
alan hodge cant mamage this section       1
Name: Description, Length: 4207, dtype: int64

From the value count, I can see that White Hanging Heart T-Light Holder is the best selling product.

In [21]:
#Changing description from object class to str
n_data = n_data[~ n_data['Description'].isna()]
n_data['Description'] = n_data['Description'].astype(str)

In [22]:
#applying the n_data into country United Kingdom
uk_data = n_data.groupby(['InvoiceNo'])['Description'].apply(list).to_frame().reset_index()

In [23]:
uk_data

,InvoiceNo,Description
0,536365,"[GLASS STAR FROSTED T-LIGHT HOLDER, SET 7 BABU..."
1,536366,"[HAND WARMER RED POLKA DOT, HAND WARMER UNION ..."
2,536367,"[HOME BUILDING BLOCK WORD, LOVE BUILDING BLOCK..."
3,536368,"[YELLOW COAT RACK PARIS FASHION, RED COAT RACK..."
4,536369,[BATH BUILDING BLOCK WORD]
...,...,...
22059,581586,"[DOORMAT RED RETROSPOT, RED RETROSPOT ROUND CA..."
22060,581587,"[BAKING SET 9 PIECE RETROSPOT , CHILDRENS APRO..."
22061,A563185,[Adjust bad debt]
22062,A563186,[Adjust bad debt]


In [24]:
transaction = uk_data['Description'].to_list()

Now that i have sorted the data using Invoice No, Product description and only to United Kingdom. I can now start building the model

In [25]:
#Setting up the association rule
association_rules = apriori(transaction, min_support=0.01, min_confidence=0.7, min_lift=3, min_length=2)

In [26]:
#Converting the associations to list
rules = list(association_rules)

In [27]:
#Printing the numbers
print(len(rules))

116


In [34]:
#Printing the rules, support, conmfidence and lift
number = 1
for rule in rules:
    pair = rule[0] 
    items = [x for x in pair]
    
    print("Rule #{}".format(number) + "\n")
    print("Antecedent: {} => Consequent: {}".format(items[0], items[1]) + "\n")
    print("Support: {}".format(str(rule[1])) + "\n")
    print("Confidence: {}".format(str(rule[2][0][2])) + "\n")
    print("Lift: {}".format(str(rule[2][0][3])) + "\n")
    print("====================================================="+"\n")
    
    print("Rule #{}".format(number))
    print("Antecedent: {} => Consequent: {}".format(items[0], items[1]))
    print("Support: {}".format(str(rule[1])))
    print("Confidence: {}".format(str(rule[2][0][2])))
    print("Lift: {}".format(str(rule[2][0][3])))
    print("=====================================================")
    number += 1

Rule #1

Antecedent: PAINTED METAL PEARS ASSORTED => Consequent: ASSORTED COLOUR BIRD ORNAMENT

Support: 0.011738578680203046

Confidence: 0.7

Lift: 10.614982817869414


Rule #1
Antecedent: PAINTED METAL PEARS ASSORTED => Consequent: ASSORTED COLOUR BIRD ORNAMENT
Support: 0.011738578680203046
Confidence: 0.7
Lift: 10.614982817869414
Rule #2

Antecedent: BAKING SET 9 PIECE RETROSPOT  => Consequent: BAKING SET SPACEBOY DESIGN

Support: 0.015137781000725162

Confidence: 0.7182795698924731

Lift: 16.968009025811057


Rule #2
Antecedent: BAKING SET 9 PIECE RETROSPOT  => Consequent: BAKING SET SPACEBOY DESIGN
Support: 0.015137781000725162
Confidence: 0.7182795698924731
Lift: 16.968009025811057
Rule #3

Antecedent: TOILET METAL SIGN => Consequent: BATHROOM METAL SIGN

Support: 0.013234227701232778

Confidence: 0.7209876543209878

Lift: 25.210573066463194


Rule #3
Antecedent: TOILET METAL SIGN => Consequent: BATHROOM METAL SIGN
Support: 0.013234227701232778
Confidence: 0.7209876543209878
Lif

With the association rules in place, I can see the items that are bought in pairs with a confidence of at least 70%.